In [2]:
# Original Author: Michael(Mike) Hann
'''
Purpose: To ingest, interpolate, filter/correct, and display Emlid Reach M2 
GNSS and Ouster Lidar OS2 data using an Extended Kalman Filter
'''

# To Do List:
#   1. create a straight interpolated dataset
#   2. develop the UKF and pass the data through
#  
#    
#     
#     


# Import Required Packages

import numpy as np
import pandas as pd
from os import chdir, getcwd
from time import perf_counter
import matplotlib.pyplot as plt


In [18]:
# INGEST FILE

# Set and Get directory
chdir(r'C:\Users\mikeh\OneDrive\Documents\GitHub\ouster_localization')
dir = getcwd()
print(f'\n\ndirectory: {dir}\n\n')

# Set the input file (full or small)
infile = 'C2_IMU.txt'
#infile = 'less_data.txt'

# Start performance counter
t1 = perf_counter()

print('\treading file...\n')
# Import the comma delimited .txt file as a pandas dataframe
df = pd.read_csv(f'{dir}\\{infile}', delimiter=',')

print('\tediting data types...\n')
# Extract only the numbers from the 'Time' column using a reg ex, convert to long integer
df['Time'] = df['Time'].str.extract('(\d+)').astype('float')
'''
t0 = df.Time[0]
print(df.Time.head())
df.Time = (df.Time-t0)/10**9
print(df.Time.head(20))
'''
# Set any future interpolated GNSS values to status 99
# Set any future interpolated standard deviations as to value of previous GNSS
df['GPS_Status'] = df['GPS_Status'].replace(' None', '99')
df.loc[:, 'SDn':'SDu'] = df.loc[:, 'SDn':'SDu'].replace(' None', pd.NA)
df.loc[:, 'SDn':'SDu'] = df.loc[:, 'SDn':'SDu'].astype('string').interpolate(method='ffill').astype(float)

# Set all values GNSS and IMU 'None' values to null, convert all object data types to presumed data types
df = df.replace(' None', pd.NA).convert_dtypes(infer_objects=True)

print(df.info())

# End performance counter
t2 = perf_counter()
print(f'time: {t2-t1}\n')



directory: C:\Users\mikeh\OneDrive\Documents\GitHub\ouster_localization


	reading file...

	editing data types...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31376 entries, 0 to 31375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Time            31376 non-null  Float64
 1   GPS_Long        1495 non-null   string 
 2   GPS_Lat         1495 non-null   string 
 3   GPS_Alt         1495 non-null   string 
 4   SDn             31376 non-null  Float64
 5   SDe             31376 non-null  Float64
 6   SDu             31376 non-null  Float64
 7   GPS_Status      31376 non-null  string 
 8   IMU_AngVelX     29881 non-null  string 
 9   IMU_AngVelY     29881 non-null  string 
 10  IMU_AngVelZ     29881 non-null  string 
 11  IMU_LinearAccX  29881 non-null  string 
 12  IMU_LinearAccY  29881 non-null  string 
 13  IMU_LinearAccZ  29881 non-null  string 
dtypes: Float64(4), string(10)
memory usage: 3.5 MB
N

In [19]:
# RE-ADJUST BAD DATA

# Start performance counter
t1 = perf_counter()

print('\tre-adjusting...\n')

l = len(df.GPS_Long)-1

# Interpolated 
ip_df = df.copy(deep=True)

'''Loop through the dataframe, if standard  
deviations are too high set that row to null'''
for i in range(1,l):    
    try:
        if ip_df.SDn[i] > 0.005:
            ip_df.loc[i,:] = pd.NA
    except:
        break
        
# Convert all values to floats
ip_df = ip_df.astype('float')
df = df.astype('float')

print(ip_df.info())

#End performance counter
t2 = perf_counter()
print(f'time: {t2-t1}')

	re-adjusting...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31376 entries, 0 to 31375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Time            27575 non-null  float64
 1   GPS_Long        1314 non-null   float64
 2   GPS_Lat         1314 non-null   float64
 3   GPS_Alt         1314 non-null   float64
 4   SDn             27575 non-null  float64
 5   SDe             27575 non-null  float64
 6   SDu             27575 non-null  float64
 7   GPS_Status      27575 non-null  float64
 8   IMU_AngVelX     26261 non-null  float64
 9   IMU_AngVelY     26261 non-null  float64
 10  IMU_AngVelZ     26261 non-null  float64
 11  IMU_LinearAccX  26261 non-null  float64
 12  IMU_LinearAccY  26261 non-null  float64
 13  IMU_LinearAccZ  26261 non-null  float64
dtypes: float64(14)
memory usage: 3.4 MB
None
time: 65.33485400000018


In [20]:
# INTERPOLATE GNSS DATA

t1 = perf_counter()

print(f'\tinterpolating "interpolated_dataframe" data...\n')

# Interpolate all values in the df as floats
ip_df = ip_df.astype('float').interpolate()

# Convert time to int, status to string, 
ip_df['GPS_Status'] = ip_df['GPS_Status'].astype('string')

# Drop first row of values (uninterpolated null IMU values)
ip_df = ip_df.drop(index=0)


print(f'\tinterpolating "standard dataframe" data...\n')

# Interpolate all values in the df as floats
df = df.astype('float').interpolate()

# Convert time to int, status to string, 
df['GPS_Status'] = df['GPS_Status'].astype('string')

# Drop first row of values (uninterpolated null IMU values)
df = df.drop(index=0)


print(df.info())

t2 = perf_counter()
print(f'time: {t2-t1}')


	interpolating "interpolated_dataframe" data...

	interpolating "standard dataframe" data...

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31375 entries, 1 to 31375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Time            31375 non-null  float64
 1   GPS_Long        31375 non-null  float64
 2   GPS_Lat         31375 non-null  float64
 3   GPS_Alt         31375 non-null  float64
 4   SDn             31375 non-null  float64
 5   SDe             31375 non-null  float64
 6   SDu             31375 non-null  float64
 7   GPS_Status      31375 non-null  string 
 8   IMU_AngVelX     31375 non-null  float64
 9   IMU_AngVelY     31375 non-null  float64
 10  IMU_AngVelZ     31375 non-null  float64
 11  IMU_LinearAccX  31375 non-null  float64
 12  IMU_LinearAccY  31375 non-null  float64
 13  IMU_LinearAccZ  31375 non-null  float64
dtypes: float64(13), string(1)
memory usage: 3.6 MB
None
time: 0.248724600000

In [21]:
# PLOT UNFILTERED DATA
%matplotlib qt

print(df.info())

plt.title("Unfiltered GNSS Tracks")
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.scatter(ip_df.GPS_Long, ip_df.GPS_Lat)
            #cmap='copper_r')
plt.scatter(df.GPS_Long, df.GPS_Lat,
           c=df.SDn, cmap='copper_r')
plt.show()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31375 entries, 1 to 31375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Time            31375 non-null  float64
 1   GPS_Long        31375 non-null  float64
 2   GPS_Lat         31375 non-null  float64
 3   GPS_Alt         31375 non-null  float64
 4   SDn             31375 non-null  float64
 5   SDe             31375 non-null  float64
 6   SDu             31375 non-null  float64
 7   GPS_Status      31375 non-null  string 
 8   IMU_AngVelX     31375 non-null  float64
 9   IMU_AngVelY     31375 non-null  float64
 10  IMU_AngVelZ     31375 non-null  float64
 11  IMU_LinearAccX  31375 non-null  float64
 12  IMU_LinearAccY  31375 non-null  float64
 13  IMU_LinearAccZ  31375 non-null  float64
dtypes: float64(13), string(1)
memory usage: 3.6 MB
None


In [23]:
# WRITE DATA TO NEW CSVs

t1 = perf_counter()

# Write data
ip_df.to_csv('del_interp_data.csv')
df.to_csv('interp_data.csv')

t2 = perf_counter()
print(f'time: {t2-t1})


In [ ]:
# PLOT RAW IMU VALUES
%matplotlib qt

# Plot Linear Acceleration   
plt.figure(figsize=(20, 5))
plt.title('IMU Linear Acceleration')
plt.plot(df.Time, df.IMU_AngVelX,
        color='red')
plt.plot(df.Time, df.IMU_AngVelY,
        color='green')
plt.plot(df.Time, df.IMU_AngVelZ,
        color='blue')
plt.xlabel('IMU Time')
plt.ylabel('g (m/s^2)')
plt.show()

# Plot Angular Velocity
plt.figure(figsize=(20, 5))
plt.title('IMU Angular Velocity')
plt.plot(df.Time, df.IMU_LinearAccX,
        color='red')
plt.plot(df.Time, df.IMU_LinearAccY,
        color='green')
plt.plot(df.Time, df.IMU_LinearAccZ,
        color='blue')
plt.xlabel('IMU Time')
plt.ylabel('deg/sec')
plt.show()

In [ ]:
# RUN DATA THROUGH THE UNSCENTED KALMAN FILTER

'''
NOT YET WORKING
'''

# Import Extended Kalman Filter class from ekf_class.py
from ukf_filterpy import UKF

# Initialize the filter
ekf = EKF(init_x=0.0, 
          init_v=1.0, 
          init_z=0.0,
          accel_variance=0.1)

# Initialize new data frame to hold filtered values



# Loop through the data frame, updating the EKF at each step
for i in range(1, len(df)):
    
    # Set change in time via IMU time
    delta_t = df.Time[i]-df.Time[i-1]
    
    # Append mean and covariance EKF values to new data frame    
    filt_df.GNSS_lat.append(ekf.gnss_mean(0))
    filt_df.GNSS_lon.append(ekf.gnss_mean(1))
    filt_df.GNSS_alt.append(ekf.gnss_mean(2))
    
    filt_df.GNSS_lat_cov.append(ekf.gnss_cov(0))
    filt_df.GNSS_lon_cov.append(ekf.gnss_cov(1))
    filt_df.GNSS_alt_cov.append(ekf.gnss_cov(2))

    # Complete the EKF predict step (code in ekf_class.py)
    ekf.predict(dt=delta_t)
    
    # Complete the EKF update step (code in ekf_class.py)
    ekf.update( meas_value=,
                meas_variance=)



In [ ]:
# PLOT FILTERED AND UNFILTERED DATA

plt.title("Filtered vs Non-filtered")
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.scatter(df.GPS_Long, df.GPS_Lat, 
            c='lightblue')
plt.scatter(filt_df.GNSS_lon, filt_df.GNSS_lat,
           c='red')
plt.show()
